In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
topping_path = 'drive/MyDrive/Data/topping_data/'
data_path = 'drive/MyDrive/Data/'

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from tqdm import tqdm
import concurrent.futures
import math
import glob
import json
import ast

## Crawl Topping

- Input file dish_data have restaurant_id an dish_id (unique for each row), with more than 1850k rows.
- Crawl topping data and keep tract of fail restaurant id (Save to fail_topping_data\*.csv with col name index is the index of the rows fail in dish_data), save data to topping_data\*.csv, merge data to toppping_data.csv
- Crawl the second time with fail index in fail_topping_data.csv file, topping_dataa*.csv, merge data to toppping_dataa.csv


### Crawl Api

In [ ]:
dish_data = pd.read_csv('drive/MyDrive/Data/DIM_SHOPEE_DISHES.csv',lineterminator='\n')
# fail_indexes_df = pd.read_csv(data_path + 'fail_topping_data.csv')

In [ ]:
headers ={
    'accept': 'application/json, text/plain, /',
    'x-foody-client-id': '525d4c83-bab2-4d1e-b17e-f867be8de94f',
    'x-foody-client-type': '4',
    'x-foody-app-type': '1004',
    'x-foody-client-version': '5.23.0',
    'x-foody-api-version': '1',
    'x-foody-client-language': 'en',
    'x-foody-access-token': '170f42e77caf17826dc30f7598aa2a56bac308fb8923a173a8fd8abfbcf53af79a5d25fe63d7deccf4b92c0384b0b718a5cf5b750b7e940bdb02507af7e38797',
    'user-agent': 'NOW/5.13.2 (sdk_gphone64_x86_64; android 33; Scale/2.75)',
    'x-foody-client-rn-version': '5.13.2',
    'x-foody-device-fingerprint': '',
    'accept-encoding': 'gzip'
}



In [ ]:
def get_topping_data(index):
    dish_index = index
    # dish_index = fail_indexes_df.iloc[index]['index']

    row = dish_data.iloc[dish_index]

    restaurant_id= round(row['dish_restaurant_id'])
    dish_id=round(row['dish_id'])
    delivery_type=1
    delivery_time='2022-11-17 00:50:00'
    shipping_type=1
    
    url = f'https://gappapi.deliverynow.vn/api/v5/buyer/store/dish/option_groups?restaurant_id={restaurant_id}&dish_id={dish_id}&delivery_type={delivery_type}&delivery_time={delivery_time}&shipping_type={shipping_type}'
    
    response = requests.get(url, headers=headers)
    topping_data = response.json()['data']['option_groups']
    if len(topping_data) > 0:
        new_row = pd.DataFrame([{'restaurant_id' : restaurant_id, 'dish_id': dish_id, 'topping_data' : topping_data}])
        return new_row
    return []

def save_file(result_df, fail_indexes, file_num):
    fail_df = pd.DataFrame({'index' : fail_indexes})
    fail_df.to_csv(topping_path + f'fail_topping_data{file_num}.csv', index=False)
    result_df.to_csv(topping_path + f'topping_data{file_num}.csv', index=False)

    # the code below is used to crawl those restaurant_id failed the first time again.
    # fail_df.to_csv(topping_path + f'fail_topping_dataa{file_num}.csv', index=False)
    # result_df.to_csv(topping_path + f'topping_dataa{file_num}.csv', index=False)

In [ ]:
notebook_num = 0 # Use for run multi notebook, each notebook have a number
                 # That number decide which are the index in dish_data to crawl
                 # Each notebook crawls 160k rows in dish_data
row_num = 160000
begin = row_num * notebook_num
end = begin + row_num
print('begin: ', begin)
print('end:  ', end)

length = end - begin
print('len: ', length)

limit = 20000 # load data to file each 20k rows.
current_limit = begin + limit
file_num = math.floor(begin/limit)
print('file_num: ', file_num)
print('current_limit: ', current_limit)



total_fail = 0

max_workers = 500 
# after several runtime test
# 500 workers is the fastest in 100, 200, 300, 500, 700, 1000 workers
# The time different of the last 3 is not significant though.


fail_indexes = []
result_df = pd.DataFrame(columns = ['restaurant_id', 'dish_id', 'topping_data'])

pbar = tqdm(total=length)

# Use Threading because those are network I/O tasks, not requiring processing power.
with concurrent.futures.ThreadPoolExecutor(max_workers= max_workers) as executor:
    index = begin

    while index < end:
        future_samples = []
        map_pool = {}

        if index >= current_limit:
            current_limit += limit
            save_file(result_df.copy(), fail_indexes.copy(), file_num)
            file_num += 1
            fail_indexes = []
            result_df = pd.DataFrame(columns = ['restaurant_id', 'dish_id', 'topping_data'])

        for i in range(max_workers):
            sample = executor.submit(get_topping_data, index)
            future_samples.append(sample)
            map_pool[sample] = index

            index += 1
            if index >= end:
                break

        for future in concurrent.futures.as_completed(future_samples):
            try:
                new_row = future.result()
                if len(new_row) > 0:
                    result_df = pd.concat([result_df, new_row], axis=0, ignore_index=True)
            except Exception as exc:
                total_fail += 1
                fail_indexes.append(map_pool[future])
            pbar.update(1)
save_file(result_df.copy(), fail_indexes.copy(), file_num)

print()
print('index:  ', index)
print('total fail: ', total_fail)


begin:  0
end:   40000
len:  40000
file_num:  0
current_limit:  10000


100%|█████████▉| 39964/40000 [24:45<00:00, 86.58it/s]


index:   40000
total fail:  8535


### Merge data to one file

In [ ]:
fail_files = glob.glob(topping_path+'fail_topping_data*.csv')
# fail_files = glob.glob(topping_path+'fail_topping_dataa*.csv')

fail_data_ls = [pd.read_csv(file) for file in fail_files]
fail_data = pd.concat(fail_data_ls, ignore_index=True)
fail_data.to_csv(data_path+'fail_topping_data.csv', index=False)
# fail_data.to_csv(data_path+'fail_topping_dataa.csv', index=False)

In [ ]:
topping_data_files = glob.glob(topping_path + 'topping_data*.csv')
# topping_data_files = glob.glob(topping_path + 'topping_dataa*.csv')

print(len(topping_data_files))
topping_data_ls = [pd.read_csv(file) for file in topping_data_files]
print(len(topping_data_ls))
topping_data_df = pd.concat(topping_data_ls, ignore_index=True)
topping_data_df

In [ ]:
topping_data_df = pd.read_csv(data_path + 'topping_data.csv')
# topping_data_df = pd.read_csv(data_path + 'topping_dataa.csv')

In [ ]:
topping_data1 = pd.read_csv(data_path + 'topping_data.csv')
topping_data1
# data crawl first time

,Unnamed: 0,restaurant_id,dish_id,topping_data
0,0,1073760,15281718,"[{'min_select': 0, 'name': 'Topping', 'rank': ..."
1,1,981894,8379142,"[{'min_select': 0, 'name': 'Chọn thêm', 'rank'..."
2,2,1114911,28885977,"[{'min_select': 0, 'name': 'Topping Trà Sữa', ..."
3,3,1073760,15281567,"[{'min_select': 0, 'name': 'Topping', 'rank': ..."
4,4,981894,8379153,"[{'min_select': 0, 'name': 'Chọn thêm', 'rank'..."
...,...,...,...,...
284678,284678,1082259,107188115,"[{'min_select': 4, 'name': 'CHỌN COMBO 4 LY', ..."
284679,284679,1082647,16436686,"[{'min_select': 0, 'name': 'Chọn sauce', 'rank..."
284680,284680,1082647,16436607,"[{'min_select': 0, 'name': 'Các Món ALC', 'ran..."
284681,284681,1082259,107188115,"[{'min_select': 4, 'name': 'CHỌN COMBO 4 LY', ..."


In [ ]:
topping_data_df
# data crawl second time

,Unnamed: 0,restaurant_id,dish_id,topping_data
0,0,4041,921141,"[{'min_select': 1, 'name': 'Chọn Size', 'rank'..."
1,1,4041,921211,"[{'min_select': 0, 'name': 'Topping', 'rank': ..."
2,2,7164,15692757,"[{'min_select': 0, 'name': 'Chọn topping', 'ra..."
3,3,7164,9126790,"[{'min_select': 0, 'name': 'Chọn topping', 'ra..."
4,4,7164,5536511,"[{'min_select': 0, 'name': 'Chọn topping', 'ra..."
...,...,...,...,...
47998,47998,1140328,86052389,"[{'min_select': 0, 'name': 'Topping', 'rank': ..."
47999,47999,1140328,86052475,"[{'min_select': 0, 'name': 'Topping', 'rank': ..."
48000,48000,1101651,19961043,"[{'min_select': 0, 'name': 'Chọn size', 'rank'..."
48001,48001,1101651,19960221,"[{'min_select': 1, 'name': 'Chọn cấp độ cay', ..."


## Transform Topping Data

In [ ]:
data = pd.concat([pd.read_csv(data_path + 'topping_dataa.csv'), pd.read_csv(data_path + 'topping_data.csv')], ignore_index=True)
data
# concat data of two crawling times.

,Unnamed: 0,restaurant_id,dish_id,topping_data
0,0,4041,921141,"[{'min_select': 1, 'name': 'Chọn Size', 'rank'..."
1,1,4041,921211,"[{'min_select': 0, 'name': 'Topping', 'rank': ..."
2,2,7164,15692757,"[{'min_select': 0, 'name': 'Chọn topping', 'ra..."
3,3,7164,9126790,"[{'min_select': 0, 'name': 'Chọn topping', 'ra..."
4,4,7164,5536511,"[{'min_select': 0, 'name': 'Chọn topping', 'ra..."
...,...,...,...,...
332681,284678,1082259,107188115,"[{'min_select': 4, 'name': 'CHỌN COMBO 4 LY', ..."
332682,284679,1082647,16436686,"[{'min_select': 0, 'name': 'Chọn sauce', 'rank..."
332683,284680,1082647,16436607,"[{'min_select': 0, 'name': 'Các Món ALC', 'ran..."
332684,284681,1082259,107188115,"[{'min_select': 4, 'name': 'CHỌN COMBO 4 LY', ..."


In [ ]:
data.drop_duplicates(inplace=True)
data

,Unnamed: 0,restaurant_id,dish_id,topping_data
0,0,4041,921141,"[{'min_select': 1, 'name': 'Chọn Size', 'rank'..."
1,1,4041,921211,"[{'min_select': 0, 'name': 'Topping', 'rank': ..."
2,2,7164,15692757,"[{'min_select': 0, 'name': 'Chọn topping', 'ra..."
3,3,7164,9126790,"[{'min_select': 0, 'name': 'Chọn topping', 'ra..."
4,4,7164,5536511,"[{'min_select': 0, 'name': 'Chọn topping', 'ra..."
...,...,...,...,...
332681,284678,1082259,107188115,"[{'min_select': 4, 'name': 'CHỌN COMBO 4 LY', ..."
332682,284679,1082647,16436686,"[{'min_select': 0, 'name': 'Chọn sauce', 'rank..."
332683,284680,1082647,16436607,"[{'min_select': 0, 'name': 'Các Món ALC', 'ran..."
332684,284681,1082259,107188115,"[{'min_select': 4, 'name': 'CHỌN COMBO 4 LY', ..."


In [ ]:
data.drop(columns='Unnamed: 0', inplace = True)
data

,restaurant_id,dish_id,topping_data
0,4041,921141,"[{'min_select': 1, 'name': 'Chọn Size', 'rank'..."
1,4041,921211,"[{'min_select': 0, 'name': 'Topping', 'rank': ..."
2,7164,15692757,"[{'min_select': 0, 'name': 'Chọn topping', 'ra..."
3,7164,9126790,"[{'min_select': 0, 'name': 'Chọn topping', 'ra..."
4,7164,5536511,"[{'min_select': 0, 'name': 'Chọn topping', 'ra..."
...,...,...,...
332681,1082259,107188115,"[{'min_select': 4, 'name': 'CHỌN COMBO 4 LY', ..."
332682,1082647,16436686,"[{'min_select': 0, 'name': 'Chọn sauce', 'rank..."
332683,1082647,16436607,"[{'min_select': 0, 'name': 'Các Món ALC', 'ran..."
332684,1082259,107188115,"[{'min_select': 4, 'name': 'CHỌN COMBO 4 LY', ..."


In [ ]:
def transform(topping_data, filename):
    # get json format fron string format
    topping_data['topping_data'] = topping_data.topping_data.apply(ast.literal_eval)

    # explode list of json_object topping_data
    topping_data = topping_data.explode('topping_data', ignore_index=True)

    # flatten json_object to pandas DataFrame
    temp = pd.json_normalize(topping_data.topping_data)

    # add prefix for new columns
    group_columns = temp.columns
    change_group_columns = {column: f'group_{column}' for column in group_columns}
    temp = temp.rename(columns = change_group_columns)

    # merge the new columns to the old ones
    topping_data = pd.concat([topping_data[['restaurant_id','dish_id']], temp], axis = 1)

    # explode list of json_object in group_options
    topping_data = topping_data.explode('group_options', ignore_index=True)
    # print(len(topping_data))

    # flatten json_object to pandas DataFrame
    temp = pd.json_normalize(topping_data['group_options'])

    # add prefix for new columns
    option_columns = temp.columns
    change_option_columns = {col: f'option_{col}' for col in option_columns}
    temp = temp.rename(columns=change_option_columns)

    # add the new columns to the old ones and then loads DataFrame to file.
    pd.concat([topping_data.loc[:, topping_data.columns != 'group_options'], temp], axis=1).to_csv(filename, index=False)
    return 1

In [ ]:
# make a folder in colab to save all the temp file data 
!mkdir file_folder
!ls

drive  file_folder  sample_data


In [ ]:
length_df = len(data)
print('data length: ', length_df)
num = 25000
print('num: ', num)
index = 0
k = 0
max_workers = 3
print('max_workers: ', max_workers)

pbar = tqdm(total = math.ceil(length_df/num))

# Can't run the whole data because it would run out of ram
# Use Multi processing because transform task require a litter more processing power.
# We dont use high number of worker because the number of processes is limited
# and it would run out of ram if we do
with concurrent.futures.ProcessPoolExecutor(max_workers= max_workers) as executor:
    begin = index

    while index < length_df:
        future_samples = []
        map_pool = {}


        for i in range(max_workers):
            print(f'Processing file {k}!')

            begin = index
            end = begin + num

            sample = executor.submit(transform, data.iloc[begin:end].copy(), f'file_folder/file{k}.csv')
            future_samples.append(sample)
            map_pool[sample] = k

            index += num
            k += 1

            if index >= length_df:
                break

        for future in concurrent.futures.as_completed(future_samples):
            try:
                a = future.result()
                print(f'Processing file {map_pool[future]} has succeed: {a}')
            except Exception as exc:
                print(exc)
                print(f'Processing file {map_pool[future]} has failed!')
            pbar.update(1)

print('Finished!')

data length:  332686
num:  25000
max_workers:  3


  0%|          | 0/14 [00:00<?, ?it/s]

Processing file 0!
Processing file 1!
Processing file 2!


  7%|▋         | 1/14 [01:36<20:57, 96.70s/it]

Processing file 1 has succeed: 1


...

Processing file 9 has succeed: 1
Processing file 12!
Processing file 13!


 93%|█████████▎| 13/14 [06:38<00:21, 21.70s/it]

Processing file 13 has succeed: 1


100%|██████████| 14/14 [06:49<00:00, 18.44s/it]

Processing file 12 has succeed: 1
Finished!


In [ ]:
!ls file_folder

file0.csv   file11.csv	file13.csv  file2.csv  file4.csv  file6.csv  file8.csv
file10.csv  file12.csv	file1.csv   file3.csv  file5.csv  file7.csv  file9.csv


In [ ]:
file_ls = glob.glob('file_folder/file*.csv')
file_ls

['file_folder/file2.csv',
 'file_folder/file12.csv',
 'file_folder/file10.csv',
 'file_folder/file1.csv',
 'file_folder/file4.csv',
 'file_folder/file7.csv',
 'file_folder/file6.csv',
 'file_folder/file11.csv',
 'file_folder/file8.csv',
 'file_folder/file0.csv',
 'file_folder/file3.csv',
 'file_folder/file13.csv',
 'file_folder/file9.csv',
 'file_folder/file5.csv']

In [ ]:
des_file = 'topping_data1.csv'

In [ ]:
for file in file_ls:
    cat = %cat {file} >> {des_file}
    print(cat)

None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [ ]:
!ls {data_path}

 1data.csv		     DIM_SHOPEE_DISHES.csv   topping_dataa.csv
 cmt_data1		     fail_topping_data.csv   topping_data.csv
'cmt_data(20k_url).csv'      topping_data	     url_40k.csv
 crawl-comment-milktea.csv   topping_data1.csv


In [ ]:
# move the final data file to google drive
!mv topping_data1.csv {data_path}

In [ ]:
#test final data
data1 = pd.read_csv(data_path + 'topping_data1.csv')
data1

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,restaurant_id,dish_id,group_min_select,group_name,group_rank,group_max_select,group_id,group_partner_option_group_id,option_name,option_weight,...,option_is_active,option_rank,option_is_default,option_max_qty,option_group_id,option_id,option_stock_info.start_time,option_stock_info.is_out_stocked,option_stock_info.end_time,option_original_price
0,1152189,118987651,1,Size,0,1,721253,NaN,250ml,1,...,True,0,True,1,721253,4682002,0,False,0,NaN
1,1152189,118987651,1,Size,0,1,721253,NaN,330ml,1,...,True,0,False,1,721253,4682003,0,False,0,NaN
2,1152189,118987651,1,Size,0,1,721253,NaN,1000ml,1,...,True,0,False,1,721253,4682004,0,False,0,NaN
3,1067920,84732845,1,Chọn size,0,1,655380,NaN,Size M,1,...,True,0,True,1,655380,4018957,0,False,0,NaN
4,1067920,84732845,1,Chọn size,0,1,655380,NaN,Size L,1,...,True,0,False,1,655380,4018958,0,False,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295909,93324,641861,1,Chọn cách xào,1,1,3941,NaN,Xào rau muống,1,...,True,7,False,1,3941,19620,0,False,0,NaN
3295910,93324,641861,1,Chọn cách xào,1,1,3941,NaN,Xào sate tép mỡ,1,...,True,8,False,1,3941,19621,0,False,0,NaN
3295911,93324,641861,1,Chọn cách xào,1,1,3941,NaN,Xào tép mỡ hành,1,...,True,9,False,1,3941,19622,0,False,0,NaN
3295912,93324,641861,1,Chọn cách xào,1,1,3941,NaN,Rang muối ớt,1,...,True,10,False,1,3941,19624,0,False,0,NaN
